In [1]:
import os
from PIL import Image, ImageDraw, ImageFont
import math

# --- CONFIGURATION ---
SOURCE_DIR = 'dataset_split/train'  # The directory containing class folders
OUTPUT_FILE = 'bharatanatyam_mudra_collage.png'
IMAGES_PER_ROW = 6                  # How many images to display per row
IMAGE_SIZE = 224                    # The size of each square image in the collage
TEXT_AREA_HEIGHT = 40               # Extra space below each image for the label
BACKGROUND_COLOR = 'white'
TEXT_COLOR = 'black'

# --- SCRIPT LOGIC ---

def create_image_collage():
    """Finds one image per class and combines them into a single PNG grid."""
    
    # 1. Find all class directories
    try:
        class_dirs = sorted([d for d in os.listdir(SOURCE_DIR) if os.path.isdir(os.path.join(SOURCE_DIR, d))])
    except FileNotFoundError:
        print(f"ERROR: The directory '{SOURCE_DIR}' was not found. Please check the path.")
        return

    num_classes = len(class_dirs)
    if num_classes == 0:
        print("ERROR: No class folders found in the source directory.")
        return
        
    print(f"Found {num_classes} classes. Preparing to create collage...")

    # 2. Calculate the dimensions for the final collage image
    num_rows = math.ceil(num_classes / IMAGES_PER_ROW)
    cell_width = IMAGE_SIZE
    cell_height = IMAGE_SIZE + TEXT_AREA_HEIGHT
    
    collage_width = IMAGES_PER_ROW * cell_width
    collage_height = num_rows * cell_height

    # 3. Create a new blank canvas for the collage
    collage = Image.new('RGB', (collage_width, collage_height), BACKGROUND_COLOR)
    draw = ImageDraw.Draw(collage)

    # Try to load a font, fall back to default if not found
    try:
        # You can change the font file path if needed, e.g., "arial.ttf"
        font = ImageFont.truetype("arial.ttf", size=16)
    except IOError:
        print("Arial font not found. Using default font.")
        font = ImageFont.load_default()

    # 4. Loop through each class, open one image, and paste it onto the canvas
    for i, class_name in enumerate(class_dirs):
        class_path = os.path.join(SOURCE_DIR, class_name)
        
        # Find the first image in the directory
        try:
            first_image_file = os.listdir(class_path)[0]
            img_path = os.path.join(class_path, first_image_file)
        except IndexError:
            print(f"Warning: No images found in '{class_path}'. Skipping.")
            continue
        
        # Open and resize the image
        img = Image.open(img_path).convert("RGB")
        img = img.resize((IMAGE_SIZE, IMAGE_SIZE))

        # Calculate the position to paste the image
        row = i // IMAGES_PER_ROW
        col = i % IMAGES_PER_ROW
        x_pos = col * cell_width
        y_pos = row * cell_height

        # Paste the image
        collage.paste(img, (x_pos, y_pos))
        
        # Add the class name as a label below the image
        text = class_name.replace('(1)', '').replace('_', ' ').strip() # Clean up the name
        text_bbox = draw.textbbox((0, 0), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_x = x_pos + (IMAGE_SIZE - text_width) // 2
        text_y = y_pos + IMAGE_SIZE + (TEXT_AREA_HEIGHT // 2) - 10

        draw.text((text_x, text_y), text, fill=TEXT_COLOR, font=font)

    # 5. Save the final collage
    collage.save(OUTPUT_FILE)
    print(f"\n🎉 Collage saved successfully to '{OUTPUT_FILE}'!")

# --- Run the function ---
if __name__ == "__main__":
    create_image_collage()

Found 29 classes. Preparing to create collage...

🎉 Collage saved successfully to 'bharatanatyam_mudra_collage.png'!
